In [1]:
import os
path = os.getcwd()

In [2]:
import numpy as np
import pandas as pd

10-1 数据探索：数据预处理

In [3]:
# 划分一次用水事件
threshold = pd.Timedelta('4 min')    # 阈值为4分钟
inputfile = path + '/data/water_heater.xls'
outputfile = path + '/tmp/dividsequence.xls'
data = pd.read_excel(inputfile)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18840 entries, 0 to 18839
Data columns (total 9 columns):
发生时间      18840 non-null int64
开关机状态     18840 non-null object
加热中       18840 non-null object
保温中       18840 non-null object
实际温度      18840 non-null object
热水量       18840 non-null object
水流量       18840 non-null int64
加热剩余时间    18840 non-null object
当前设置温度    18840 non-null object
dtypes: int64(2), object(7)
memory usage: 1.3+ MB


In [4]:
data.head()

,发生时间,开关机状态,加热中,保温中,实际温度,热水量,水流量,加热剩余时间,当前设置温度
0,20141019063917,关,关,关,30°C,0%,0,0分钟,50°C
1,20141019070154,关,关,关,30°C,0%,0,0分钟,50°C
2,20141019070156,关,关,关,30°C,0%,8,0分钟,50°C
3,20141019071230,关,关,关,30°C,0%,0,0分钟,50°C
4,20141019071236,关,关,关,29°C,0%,0,0分钟,50°C


In [5]:
# dataframe处理
data[u'发生时间'] = pd.to_datetime(data[u'发生时间'], format = '%Y%m%d%H%M%S')
data = data[data[u'水流量'] > 0]    # 取流量大于0的数据
d = data[u'发生时间'].diff() > threshold    # 相邻时间作差分，大于threshold
data[u'事件编号'] = d.cumsum() + 1    # 通过累积求和的方式为事件编号
data.to_excel(outputfile)
data.head()

,发生时间,开关机状态,加热中,保温中,实际温度,热水量,水流量,加热剩余时间,当前设置温度,事件编号
2,2014-10-19 07:01:56,关,关,关,30°C,0%,8,0分钟,50°C,1
56,2014-10-19 07:38:16,关,关,关,30°C,0%,8,0分钟,50°C,2
381,2014-10-19 09:46:38,关,关,关,29°C,0%,16,0分钟,50°C,3
382,2014-10-19 09:46:40,关,关,关,29°C,0%,13,0分钟,50°C,3
384,2014-10-19 09:47:15,关,关,关,29°C,0%,20,0分钟,50°C,3


10-2 阈值寻优：划分用水事件的最优时间间隔 

In [6]:
n = 4    # 使用之后四个点的平均斜率

threshold = pd.Timedelta(minutes = 5)    # 专家阈值
data = pd.read_excel(inputfile)
data[u'发生时间'] = pd.to_datetime(data[u'发生时间'], format = '%Y%m%d%H%M%S')
data = data[data[u'水流量'] > 0]    # 只需流量大于0的记录

def event_num(ts):
    d = data[u'发生时间'].diff() > ts    # 相邻时间差分 > 阈值
    return d.sum() + 1    # 返回事件数

# 定义阈值列
dt = [pd.Timedelta(minutes = i) for i in np.arange(1, 9, 0.25)]
h = pd.DataFrame(dt, columns = [u'阈值'])    # 定义阈值列
h[u'事件数'] = h[u'阈值'].apply(event_num)   # 计算每个阈值对应的事件数
h[u'斜率'] = h[u'事件数'].diff() / 0.25      # 计算每两个相邻点对应的斜率
h[u'斜率指标'] = pd.Series.rolling(h[u'斜率'].abs(), n).mean()  # 采用后n个的斜率绝对值平均作为斜率指标
ts = h[u'阈值'][h[u'斜率指标'].idxmin() - n] # 返回斜率指标最小值对应的阈值
h

,阈值,事件数,斜率,斜率指标
0,00:01:00,232,NaN,NaN
1,00:01:15,227,-20.0,NaN
2,00:01:30,218,-36.0,NaN
3,00:01:45,207,-44.0,NaN
4,00:02:00,201,-24.0,31.0
5,00:02:15,197,-16.0,30.0
6,00:02:30,194,-12.0,24.0
7,00:02:45,191,-12.0,16.0
8,00:03:00,186,-20.0,15.0
9,00:03:15,181,-20.0,16.0


In [7]:
ts

Timedelta('0 days 00:04:00')

In [8]:
if ts > threshold:
    ts = pd.Timedelta(minutes = 4)
print(ts)

0 days 00:04:00


10-3 训练多层神经网络模型

In [9]:
from keras.models import Sequential
from keras.layers.core import Activation, Dense

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
inputfile1 = path + '/data/train_neural_network_data.xls'  # 训练数据
inputfile2 = path + '/data/test_neural_network_data.xls'   # 测试数据
testoutputfile = path + '/tmp/test_output_data.xls'        # 测试数据输出

# 读取训练集和测试集
data_train = pd.read_excel(inputfile1)
data_test = pd.read_excel(inputfile2)
data_train.head()

,热水事件,起始数据编号,终止数据编号,开始时间（begin_time）,根据日志判断是否为洗浴（1表示是，0表示否）,洗浴时间点,总用水时长（w_time）,总停顿时长（w_pause_time）,平均停顿时长（avg_pause_time）,停顿次数（pause）,用水时长（use_water_time）,用水/总时长（use_water_rate）,总用水量（w_water）,平均水流量（water_rate）,水流量波动（flow_volatility）,停顿时长波动（pause_volatility）
0,1,218,344,2014-10-19 08:51:30',0,8,591.999998,303.500013,50.583336,6,288.499985,0.487331,12.998333,2.703293,0.870856,650.106848
1,2,569,965,2014-10-19 15:55:23',1,15,1008.000005,46.499999,46.499999,1,961.500006,0.953869,50.626667,3.159230,0.202300,0.000000
2,3,1077,1128,2014-10-19 18:21:40',0,18,467.999997,269.499987,53.899997,5,198.500010,0.424145,7.087500,2.142317,0.404960,531.384976
3,4,1973,2236,2014-10-20 16:42:41',1,16,660.999996,23.499993,23.499993,1,637.500003,0.964448,32.193333,3.029961,0.291306,0.000000
4,5,2320,2435,2014-10-20 18:05:28',1,18,550.000005,164.500013,32.900003,5,385.499992,0.700909,13.459167,2.094812,0.395200,180.384977


In [11]:
data_test.head()

,热水事件,起始数据编号,终止数据编号,开始时间（begin_time）,根据日志判断是否为洗浴（1表示是，0表示否）,洗浴时间点,总用水时长（w_time）,总停顿时长（w_pause_time）,平均停顿时长（avg_pause_time）,停顿次数（pause）,用水时长（use_water_time）,用水/总时长（use_water_rate）,总用水量（w_water）,平均水流量（water_rate）,水流量波动（flow_volatility）,停顿时长波动（pause_volatility）
0,1,73,336,2015-01-05 9:42:41',1,9,660.999996,23.499993,23.499993,1,637.500003,0.964448,32.193333,3.029961,0.291306,0.000000e+00
1,2,420,535,'2015-01-05 18:05:28',1,18,550.000005,164.500013,32.900003,5,385.499992,0.700909,13.459167,2.094812,0.395200,1.803850e+02
2,3,538,706,'2015-01-05 18:25:24',1,18,649.000001,201.000006,201.000006,1,447.999995,0.690293,22.622500,3.029799,0.643431,0.000000e+00
3,4,793,910,'2015-01-05 20:00:42',1,20,297.999994,7.999977,1.999994,4,290.000017,0.973154,15.088333,3.121724,1.075976,2.529227e-11
4,5,935,1133,'2015-01-05 20:15:13',1,20,623.999996,4.999993,4.999993,1,619.000003,0.991987,41.015000,3.975606,0.202620,0.000000e+00


In [12]:
# 划分样本特征和标签
y_train = data_train.iloc[:, 4].as_matrix()
x_train = data_train.iloc[:, 5:17].as_matrix()
y_test = data_test.iloc[:, 4].as_matrix()
x_test = data_test.iloc[:, 5:17].as_matrix()

# 建立神经网络模型
model = Sequential()
model.add(Dense(input_dim = 11, units = 17))
model.add(Activation('relu'))
model.add(Dense(input_dim = 17, units = 10))
model.add(Activation('relu'))
model.add(Dense(input_dim = 10, units = 1))
model.add(Activation('sigmoid'))
# 编译模型
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    sample_weight_mode = 'binary'
)
# 训练模型
model.fit(x_train, y_train, nb_epoch = 500, batch_size = 1)
# 保存模型
model.save_weights(path + '/tmp/net.model')

# 进行预测
r = pd.DataFrame(model.predict_classes(x_test), columns = [u'预测结果'])
pd.concat([data_test.iloc[:, :5], r], axis = 1).to_excel(testoutputfile)
y_model = model.predict(x_test)
y_model

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/500
28/28 [==============================] - 1s 53ms/step - loss: 4.0044
Epoch 2/500
28/28 [==============================] - 0s 5ms/step - loss: 4.0044
Epoch 3/500
28/28 [==============================] - 0s 4ms/step - loss: 4.0223
Epoch 4/500
28/28 [==============================] - 0s 5ms/step - loss: 4.3948
Epoch 5/500
28/28 [==============================] - 0s 4ms/step - loss: 3.4339
Epoch 6/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8668
Epoch 7/500
28/28 [==============================] - 0s 4ms/step - loss: 2.9256
Epoch 8/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8708
Epoch 9/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8629
Epoch 10/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8532
Epoch 11/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8532
Epoch 12/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8532
Epoch 13/500
28/28 [================

28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 103/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 104/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 105/500
28/28 [==============================] - 0s 6ms/step - loss: 2.8531
Epoch 106/500
28/28 [==============================] - 0s 15ms/step - loss: 2.8531
Epoch 107/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 108/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 109/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 110/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 111/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 112/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 113/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 114/500
28/28 [========

D:\Anaconda3\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.233056). Check your callbacks.
  % delta_t_median)
D:\Anaconda3\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.138775). Check your callbacks.
  % delta_t_median)


28/28 [==============================] - 0s 17ms/step - loss: 2.8531
Epoch 137/500
28/28 [==============================] - 0s 6ms/step - loss: 2.8531
Epoch 138/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 139/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 140/500
28/28 [==============================] - 0s 6ms/step - loss: 2.8531
Epoch 141/500
28/28 [==============================] - 0s 7ms/step - loss: 2.8531
Epoch 142/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 143/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 144/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 145/500
28/28 [==============================] - 0s 6ms/step - loss: 2.8531
Epoch 146/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 147/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 148/500
28/28 [========

D:\Anaconda3\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.137074). Check your callbacks.
  % delta_t_median)


28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 232/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 233/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 234/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 235/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 236/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 237/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 238/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 239/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 240/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 241/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 242/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 243/500
28/28 [=========

28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 332/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 333/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 334/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 335/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 336/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 337/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 338/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 339/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 340/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 341/500
28/28 [==============================] - 0s 4ms/step - loss: 2.8531
Epoch 342/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 343/500
28/28 [=========

28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 432/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 433/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 434/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 435/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 436/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 437/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 438/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 439/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 440/500
28/28 [==============================] - 0s 6ms/step - loss: 2.8531
Epoch 441/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 442/500
28/28 [==============================] - 0s 5ms/step - loss: 2.8531
Epoch 443/500
28/28 [=========

array([[1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.9999845e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [6.7116205e-35],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00]], dtype=float32)

In [13]:
r

,预测结果
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


10-4 模型检验

In [14]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, r))

0.7619047619047619


10-5 连续洗浴事件识别

In [15]:
outputfile = path + '/tmp/attribute_extract.xls'
data = pd.read_excel(inputfile)

data[u'发生时间'] = pd.to_datetime(data[u'发生时间'], format = '%Y%m%d%H%M%S')
data = data[data[u'水流量'] > 0]
d = data[u'发生时间'].diff() > threshold
data[u'事件编号'] = d.cumsum() + 1
data.head()

,发生时间,开关机状态,加热中,保温中,实际温度,热水量,水流量,加热剩余时间,当前设置温度,事件编号
2,2014-10-19 07:01:56,关,关,关,30°C,0%,8,0分钟,50°C,1
56,2014-10-19 07:38:16,关,关,关,30°C,0%,8,0分钟,50°C,2
381,2014-10-19 09:46:38,关,关,关,29°C,0%,16,0分钟,50°C,3
382,2014-10-19 09:46:40,关,关,关,29°C,0%,13,0分钟,50°C,3
384,2014-10-19 09:47:15,关,关,关,29°C,0%,20,0分钟,50°C,3


In [16]:
data_g = data.groupby(u'事件编号')
result = pd.DataFrame()
dt = pd.Timedelta(seconds = 2)
data_g.head()

,发生时间,开关机状态,加热中,保温中,实际温度,热水量,水流量,加热剩余时间,当前设置温度,事件编号
2,2014-10-19 07:01:56,关,关,关,30°C,0%,8,0分钟,50°C,1
56,2014-10-19 07:38:16,关,关,关,30°C,0%,8,0分钟,50°C,2
381,2014-10-19 09:46:38,关,关,关,29°C,0%,16,0分钟,50°C,3
382,2014-10-19 09:46:40,关,关,关,29°C,0%,13,0分钟,50°C,3
384,2014-10-19 09:47:15,关,关,关,29°C,0%,20,0分钟,50°C,3
404,2014-10-19 11:50:17,关,关,关,29°C,0%,22,0分钟,50°C,4
407,2014-10-19 13:56:21,关,关,关,29°C,0%,8,0分钟,50°C,5
410,2014-10-19 15:34:39,关,关,关,29°C,0%,36,0分钟,50°C,6
411,2014-10-19 15:34:40,关,关,关,29°C,0%,32,0分钟,50°C,6
412,2014-10-19 15:34:45,关,关,关,29°C,0%,31,0分钟,50°C,6


In [17]:
for _, g in data_g:
    temp = pd.DataFrame(index = [0])
    # 根据用水时长、开关机切换次数、总用水量推出是否是洗澡
    tstart = g[u'发生时间'].min()
    tend = g[u'发生时间'].max()
    temp[u'用水事件时长（M）'] = (dt + tend - tstart).total_seconds() / 60
    temp[u'开关机切换次数'] = (pd.Series.rolling(g[u'开关机状态'] == u'关', 2).sum() == 1).sum()
    temp[u'总用水量（L）'] = g[u'水流量'].sum()
    tdiff = g[u'发生时间'].diff()
    if len(g[u'发生时间']) == 1:
        temp[u'总用水时长（Min）'] = dt.total_seconds() / 60
    else:
        temp[u'总用水时长（Min）'] = (
            tdiff.sum() - tdiff.iloc[1] / 2 -
            tdiff.iloc[len(tdiff) - 1] / 2
        ).total_seconds() / 60
    temp[u'平均水流量（L/min）'] = temp[u'总用水量（L）'] / temp[u'总用水时长（Min）']
    result = result.append(temp, ignore_index = True)
result.to_excel(outputfile)

In [18]:
result.head()

,用水事件时长（M）,开关机切换次数,总用水量（L）,总用水时长（Min）,平均水流量（L/min）
0,0.033333,0,8,0.033333,240.000000
1,0.033333,0,8,0.033333,240.000000
2,0.650000,0,49,0.308333,158.918919
3,0.033333,0,22,0.033333,660.000000
4,0.033333,0,8,0.033333,240.000000
